In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import tensorflow as tf, numpy as np, os, sys
p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

In [2]:
# TODO: adjust imports

from cnn_sys_ident.architectures.models import BaseModel, CorePlusReadoutModel
from cnn_sys_ident.architectures.cores import StackedRotEquiHermiteConv2dCore
from cnn_sys_ident.architectures.readouts import SpatialXFeatureJointL1Readout
from cnn_sys_ident.architectures.training import Trainer
from cnn_sys_ident.retina.data import Dataset
from cnn_sys_ident.retina.read_data import MultiDatasetWrapper

# Parameters

In [3]:
# Training
VAL_STEPS = 50
LEARNING_RATE = 0.002
BATCH_SIZE = 256
PATIENCE = 5
LR_DECAY_STEPS = 2

# Linear model processing movies

In [5]:
md_wrapper = MultiDatasetWrapper(experimenter="Franke",
                                 date="2018-10-19",
                                 exp_num=1,
                                 stim_path='/gpfs01/euler/User/lhoefling/Stimuli/MouseCam2/')
md_wrapper.generate_dataset()
data = md_wrapper.multi_dataset # TODO: data loading
base = BaseModel(
    data,
    log_hash='testing'   # If you omit this, it will generate a new, random hash every time
)
core = MultiScanCore(
    base,
    base.inputs,
    core_type=StackedFactorizedConv3dCore,
    filter_size_spatial=[30],   # probably too large -- otherwise downsample movies
    filter_size_temporal=[20],
    num_filters=[4],
    activation_fn=None,
)
readout = MultiScanReadout(
    base,
    core.output,
    readout_type=SpatialXFeature3dL1Readout,
    mask_sparsity=0.01,     # TO DO: find good value
    feature_sparsity=0.001, # TO DO: find good value
    positive_feature_weights=False,
)
model = CorePlusReadoutModel(base, core, readout)
model.load()
trainer = Trainer(base, model)
iter_num, val_loss, test_corr = trainer.fit(
    val_steps=VAL_STEPS,
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    patience=PATIENCE,
    lr_decay_steps=LR_DECAY_STEPS)

trainer.compute_test_corr()      #   (for training, see below)

Connecting lhoefling@172.25.240.205:3306


KeyboardInterrupt: 